In [1]:
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
from huggingface_hub import login
import bitsandbytes as bnb

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["TRANSFORMERS_CACHE"] = "./model_cache"

tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    cache_dir="./model_cache",
    trust_remote_code=True,
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    device_map="auto",
    load_in_8bit=True,
    trust_remote_code=True,
    cache_dir="./model_cache",
    use_auth_token=os.getenv("HF_TOKEN")
)


In [10]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
  

In [13]:
model.save_pretrained("./quantized_mistral_7b_model")
tokenizer.save_pretrained("./quantized_mistral_7b_model")


('./quantized_mistral_7b_model\\tokenizer_config.json',
 './quantized_mistral_7b_model\\special_tokens_map.json',
 './quantized_mistral_7b_model\\tokenizer.model',
 './quantized_mistral_7b_model\\added_tokens.json',
 './quantized_mistral_7b_model\\tokenizer.json')

In [3]:
model_dir = "./quantized_mistral_7b_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="auto", load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
c:\Python_Environ\envs\new_env\lib\site-packages\transformers\quantizers\auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def process_agronomist_message(raw_message):
    system_prompt = """Ты являешься помощником для занесения данных о выполненных сельскохозяйственных работах в таблицу."""

    user_prompt = f"""
        <Инструкция>
    Сложность состоит в том, что
    в своих отчетах агрономы используют различные сокращения, а в итоговой таблице необходимо указать полное название работы.
    Чтобы ты смог выполнить эту задачу, я дам тебе справочную информацию (полные названия работ и всех наименовай, которые встретятся в отчетах сотрудников). Она будет приведена ниже.
    В итоговой таблице 6 обязательных столбцов, соответственно, в отчете сотрудников должны присутствовать как минимум 6 наименований. Отдельно отмечу, что в сообщении чаще всего может быть сразу несколько описаний работ,
    то есть агроном мог выполнять несколько работ за день и описать их в одном сообщении. Обычно в таком случае эти описания разделены пустыми строками, но могут и не быть. Если описаний несколько, то дополнительно сообщие мне об этом и укажи точное количество.
    В этом случае ты должен будешь указать все 6 обязательных полей для каждой работы.
    Как я уже сказал, обязательных полей всего 6, но всего их 8.


    Далее ниже будут приведены 6 обязательных полей, значения для которых
    ОБЯЗАТЕЛЬНО должны присутствовать в таблице. Затем для первых трех полей (Подразделение, Операция, Культура) будет приведена справочная информация. В конце инструкции будут приведены два примера входных сообщений от агронома (в формате JSON) и пример вывода в итоговую таблицу.

     6 обязательных полей:
    'Дата' -- Дата выполнения работы (в формате MM-DD). Извлекается из сообщения агронома, если в нем есть. Если нет, то извлекается из поля 'date' (в формате YYYY-MM-DD-Time-zone) и выводится в таблице.
    'Подразделение' -- Принадлежность отделений и производственных участков (ПУ) к подразделениям
    'Операция' -- Наименование выполняемой работы
    'Культура' -- Наименование культуры, на которой выполнялась работа
    'За день га' -- Площадь, на которой выполнялась работа (в гектарах)
    'C начала операции' -- Площадь, на которой выполнялась работа (в гектарах) с начала операции

    Необязательные поля (информации о них может не быть в сообщении агронома):

    'Вал за день, ц' -- количество собранной сельскохозяйственной культуры за день (в центнерах)
    'Вал с начала, ц' -- количество собранной сельскохозяйственной культуры за весь период (в центнерах)

    Если нет значений для полей 'Вал за день, ц' и 'Вал с начала, ц', то оставляй их пустыми. Если они есть, то вноси их в таблицу и ОБЯЗАТЕЛЬНО ставь запятую перед двумя последними цифрами, так как агрономы присылают эти значения в кг, а нужно выводить в центнерах.
    Вносить их нужно ТОЛЬКО в том случае, если они присутствуют в сообщении агронома.

    Справочная информация:

    1. Подразделение

| Подразделение       | Производственный участок (ПУ) | № Отделения      |
|---------------------|-------------------------------|------------------|
| АОР                 | Кавказ                        | 18               |
| АОР                 | Кавказ                        | 19               |
| АОР                 | Север                         | 3                |
| АОР                 | Север                         | 7                |
| АОР                 | Север                         | 10               |
| АОР                 | Север                         | 20               |
| АОР                 | Центр                         | 1                |
| АОР                 | Центр                         | 4                |
| АОР                 | Центр                         | 5                |
| АОР                 | Центр                         | 6                |
| АОР                 | Центр                         | 9                |
| АОР                 | Юг                            | 11               |
| АОР                 | Юг                            | 12               |
| АОР                 | Юг                            | 16               |
| АОР                 | Юг                            | 17               |
| АОР                 | Рассвет                       |                  |
| ТСК                 | Нет ПУ                        | Нет отделения    |
| АО Кропоткинское    | Нет ПУ                        | Нет отделения    |
| Восход              | Нет ПУ                        | Нет отделения    |
| Колхоз Прогресс     | Нет ПУ                        | Нет отделения    |
| Мир                 | Нет ПУ                        | Нет отделения    |
| СП Коломейцево      | Нет ПУ                        | Нет отделения    |

2. Операция

| Наименования полевых работ        | Примечание                                         |
|-----------------------------------|----------------------------------------------------|
| 1-я междурядная культивация       | На всех культурах кроме пшеницы, ячменя            |
| 2-я междурядная культивация       | На всех культурах кроме пшеницы, ячменя            |
| Боронование довсходовое           |                                                    |
| Внесение минеральных удобрений    |                                                    |
| Выравнивание зяби                 |                                                    |
| 2-е Выравнивание зяби             |                                                    |
| Гербицидная обработка             | На свекле их 4 шт, на остальных культурах 1        |
| 1 Гербицидная обработка           |                                                    |
| 2 Гербицидная обработка           |                                                    |
| 3 Гербицидная обработка           |                                                    |
| 4 Гербицидная обработка           |                                                    |
| Дискование                        |                                                    |
| Дискование 2-е                    |                                                    |
| Дискование 3-е                    |                                                    |
| Инсектицидная обработка           |                                                    |
| Культивация                       |                                                    |
| Пахота                            |                                                    |
| Подкормка                         |                                                    |
| 2-я подкормка                     |                                                    |
| Предпосевная культивация          |                                                    |
| Прикатывание посевов              |                                                    |
| Сев                               |                                                    |
| Сплошная культивация              |                                                    |
| Уборка                            |                                                    |
| Функицидная обработка             |                                                    |
| Чизлевание                        |                                                    |

3. Культура

| Наименования с/х культур                |
|----------------------------------------|
| Вика+Тритикале                         |
| Горох на зерно                         |
| Горох товарный                         |
| Гуар                                   |
| Конопля                                |
| Кориандр                               |
| Кукуруза кормовая                      |
| Кукуруза семенная                      |
| Кукуруза товарная                      |
| Люцерна                                |
| Многолетние злаковые травы            |
| Многолетние травы прошлых лет         |
| Многолетние травы текущего года       |
| Овес                                   |
| Подсолнечник кондитерский             |
| Подсолнечник семенной                 |
| Подсолнечник товарный                 |
| Просо                                 |
| Пшеница озимая на зеленый корм        |
| Пшеница озимая семенная               |
| Пшеница озимая товарная               |
| Рапс озимый                            |
| Рапс яровой                            |
| Свекла сахарная                        |
| Сорго                                  |
| Сорго кормовой                         |
| Сорго-суданковый гибрид                |
| Соя семенная                           |
| Соя товарная                           |
| Чистый пар                             |
| Чумиза                                 |
| Ячмень озимый                          |
| Ячмень озимый семенной                |

Первый пример входного сообщения от агронома:


    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender":
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"

    "reply_to_message_id": null,
    "raw_text": "Пахота зяби под мн тр
По Пу 26/488
Отд 12 26/221

Предп культ под оз пш
По Пу 215/1015
Отд 12 128/317
Отд 16 123/529

2-е диск сах св под пш
По Пу 22/627
Отд 11 22/217

2-е диск сои под оз пш
По Пу 45/1907
Отд 12 45/299"


Пример вывода:

| Подразделение | Вид работ                | С/х культура                   | № поля | Площадь (га) |
|---------------|--------------------------|--------------------------------|--------|--------------|
| АОР           | Пахота                   | Многолетние травы текущего года | 26     | 488          |
| АОР           | Предпосевная культивация | Пшеница озимая товарная        | 215    | 1015         |
| АОР           | Дискование 2-е           | Пшеница озимая товарная        | 22     | 627          |
| АОР           | Дискование 2-е           | Пшеница озимая товарная        | 45     | 1907         |

Отдельно замечу, что если в сообщении есть фраза 'По ПУ' или 'По Пу', то в поля '№ поля' и 'Площадь (га)' необходимо ОБЯЗАТЕЛЬНО вносить значения, которые следует после этой фразы,
 они приоритетнее тех, которые указаны непосредственно строкой ниже. 


Второй пример входного сообщения от агронома:


    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender":
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    ,
    "reply_to_message_id": null,
    "raw_text": "Уборка свеклы 27.10.день
Отд10-45/216
По ПУ 45/1569
Вал 1259680/6660630
Урожайность 279,9/308,3
По ПУ 1259680/41630600
На завод 1811630/6430580
По ПУ 1811630/41400550
Положено в кагат 399400
Вввезено с кагата 951340
Остаток 230060
Оз-9,04/12,58
Дигестия-14,50/15,05"



Таблица:

| Дата       | Подразделение | Операция | Культура         | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|----------|------------------|-------------|------------------------|----------------|------------------|
| 10/27/2024 | АОР            | Уборка   | Свекла сахарная  | 45          | 1 569                  | 12 596,80      | 66 606,30        |


    Проанализируй следующее сообщение от агронома и преобразуй его в структурированную таблицу в соответствии с приведенной выше инструкцией.
    Учти, что из сообщения можно извлечь только 5 обязательных полей, а остальные поля могут отсутствовать. Но при этом если есть все 8 полей,
    то они тоже должны быть указаны в таблице. Поле 'Дата' ты либо извлекаешь из сообщения, либо берешь из поля 'date' (в формате YYYY-MM-DD-Time-zone).

    {raw_message}

    Верни ответ в виде таблицы в формате markdown, где каждая строка представляет одну операцию. Укажи количество найденных операций. В конце ОБЯЗАТЕЛЬНО проверь,
    что все занесенные тобой в таблицу данные корректны и соответствуют наименованиям из справочной информации. Если не соответсвуют, то исправь.

    Учти исключения:

    Если в сообщении есть фраза 'Выкаш отц форм под/г', то данную строку операцию учитывать не нужно, в итоговой таблице ее быть НЕ должно.


    В ответе выведи ТОЛЬКО  таблицу в виде markdown
    </Инструкция>
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=2000,
            temperature=0.3,
            top_p=0.8,
            do_sample=True
        )
    
 
    response = tokenizer.decode(output[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response



In [6]:
if __name__ == "__main__":
    raw_message = """
    "date": "2025-04-13T12:26:25+03:00",
    "date_utc": "2025-04-13T09:26:25+00:00",
    "tg_message_id": 10,
    "our_message_id": 1,
    "sender":
        "id": 67257831,
        "first_name": "Vadim",
        "last_name": "Palyonov",
        "username": "vAD_lv"
    ,
    "reply_to_message_id": null,
    "raw_text": "Пахота под сах св
По Пу 77/518
Отд 12 46/298
Отд 16 21/143
Отд 17 10/17

Чизел под оз ячмень 
По Пу 22/640
Отд 11 22/242

Чизел под оз зел корм
Отд 11 40/40

Диск оз пшеницы
По Пу 28/8872
Отд 17 28/2097

2-е диск под сах св
По Пу 189/1763
Отд 11 60/209
Отд 12 122/540
Отд 17 7/172

Диск кук силос
По Пу 6/904
Отд 11 6/229

Прик под оз ячмень
По Пу 40/498
Отд 11 40/100

Уборка сои (семенной)
Отд 11 65/65
Вал 58720
Урож 9"
    """

    result = process_agronomist_message(raw_message)
    print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


| Дата       | Подразделение | Операция                 | Культура         | За день, га | С начала операции, га | Вал за день, ц | Вал с начала, ц |
|------------|----------------|--------------------------|------------------|-------------|------------------------|----------------|------------------|
| 2025-04-13  | АОР            | Пахота под сах св          | Свекла сахарная  | 77          | 77                     |                |                |
| 2025-04-13  | АОР            | Чизел под оз ячмень       | Ячмень озимый     | 22          | 22                     |                |                |
| 2025-04-13  | АОР            | Чизел под оз зел корм      | Зернобобовые      | 40          | 40                     |                |                |
| 2025-04-13  | АОР            | Диск оз пшеницы          | Пшеница озимая     | 28          | 28                     |                |                |
| 2025-04-13  | АОР            | 2-е диск под сах св        | Свекла сахарная  |